In [ ]:
import numpy as np
import pandas as pd
import tifffile
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.metrics import jaccard_score
from scipy.spatial.distance import directed_hausdorff

###### Write utility functions

In [ ]:
def enc2mask(encs, shape):
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for m, enc in enumerate(encs):
        if isinstance(enc, np.float) and np.isnan(enc):
            continue
        enc_split = enc.split()
        for i in range(len(enc_split) // 2):
            start = int(enc_split[2 * i]) - 1
            length = int(enc_split[2 * i + 1])
            img[start: start + length] = 1 + m

    return img.reshape(shape).T

In [ ]:
def dice_scores_img(pred, truth, eps=1e-8):
    pred = pred.reshape(-1) > 0
    truth = truth.reshape(-1) > 0
    intersect = (pred & truth).sum(-1)
    union = pred.sum(-1) + truth.sum(-1)

    dice = (2.0 * intersect + eps) / (union + eps)
    return dice

In [ ]:
def perf_metrics(gt, pred):
    n = 0
    d = 0
    for i in range(gt.shape[0]):
        for j in range (gt.shape[1]):
            if (gt[i][j]==pred[i][j]):
                n = n+1
            d = d+1
    
    return n/d, jaccard_score(gt.flatten(order='C'), pred.flatten(order='C')), directed_hausdorff(gt, pred)

<h6> Calculate mean metrics values for test images </h6> 

In [ ]:
DATA_PATH = Path('path/to/dataset/')
df_pred = pd.read_csv('predictions.csv')
df_truth = pd.read_csv(DATA_PATH/'ground_truth.csv')
df_info = pd.read_csv(DATA_PATH/'metadata.csv')

In [ ]:
df_pred

In [ ]:
plt.rcParams["figure.figsize"] = [20,20]

In [ ]:
scores = []
pa_list = []
ji_list = []
haus_dis_list = []

for img in df_truth['filename'].unique():
    
    organ = df_truth[df_truth['filename'] == img]['tissue_name']
    im_orig = tifffile.imread(str(DATA_PATH)+"/test/"+img+".tif")
    
    shape_ = im_orig.shape
    shape = (shape_[0],shape_[1])#df_info[df_info.image_file == img][['width_pixels', 'height_pixels']].values.astype(int)[0]
    truth = df_truth[df_truth['filename'] == img]['rle']
    mask_truth = enc2mask(truth, shape)
    print(img)
    pred = df_pred[df_pred['filename'] == img]['predicted']
    mask_pred = enc2mask(pred, shape)
    fig,ax = plt.subplots(1,3)
    # plt.title(f"Organ {organ.values[0]}")
    # plt.suptitle()
    # fig.tight_layout()

    ax[0].imshow(im_orig)
    ax[0].set_title(f"Image Organ = {organ.values[0]}")
    ax[1].imshow(mask_truth)
    ax[1].set_title("Ground truth")
    ax[2].imshow(mask_pred)
    ax[2].set_title("Predicted mask")
    plt.show()
    score = dice_scores_img(mask_pred, mask_truth)
    print (score)
#     pa, ji, haus = perf_metrics(mask_pred, mask_truth)    
#     pa_list.append (pa)
#     ji_list.append(ji)
#     haus_dis_list.append(haus[0])
    scores.append(score)

In [ ]:
l = len(df_pred)
for img, s in zip(df_truth[5:]['filename'],scores):
    print (round(s, 3))
    
print ("Average Dice Score = ", round(sum(scores)/l,3))

In [ ]:
df_pred['dice_scores'] = scores

In [ ]:
df_pred.to_csv("test_result_dice.csv",index=None)